In [1]:
import pandas as pd
import os
import nibabel as nib

# Define a function to extract ROIs around microbleed locations
def extract_roi(swi_data, coordinates, roi_size):
    rois = []
    for coord in coordinates:
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]
        rois.append(roi)
    return rois

# Load the Excel file containing microbleed coordinates
excel_file = 'NoCMB.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'noncmb'  # Replace 'path_to_rCMB_folder' with your folder path

# Iterate through rows to load NIFTI files and extract microbleed locations
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        print(f"NIFTI Filename: {nifti_filename}")
        print(f"Microbleed Coordinates: {microbleed_coordinates}")
        print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Extract ROIs around microbleed locations
        microbleed_rois = extract_roi(swi_data, microbleed_coordinates, roi_size)

        # Display information about the extracted ROIs
        for i, roi in enumerate(microbleed_rois):
            print(f"Microbleed {i+1} - ROI Shape: {roi.shape}")

        print()  # Separate output for each NIFTI file
    else:
        print(f"NIFTI file {nifti_filename} not found in the folder.")


C:\Users\Dell\AppData\Local\Temp\ipykernel_10308\3580032934.py:44: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_10308\3580032934.py:45: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_10308\3580032934.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(

NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(62, 105, 53), (51, 133, 60), (131, 121, 21), (43, 149, 61), (105, 93, 51), (103, 208, 27), (52, 150, 40), (46, 181, 17), (116, 86, 50), (143, 129, 42)]
SWI Image Shape: (176, 256, 80)
Microbleed 1 - ROI Shape: (10, 10, 10)
Microbleed 2 - ROI Shape: (10, 10, 10)
Microbleed 3 - ROI Shape: (10, 10, 10)
Microbleed 4 - ROI Shape: (10, 10, 10)
Microbleed 5 - ROI Shape: (10, 10, 10)
Microbleed 6 - ROI Shape: (10, 10, 10)
Microbleed 7 - ROI Shape: (10, 10, 10)
Microbleed 8 - ROI Shape: (10, 10, 10)
Microbleed 9 - ROI Shape: (10, 10, 10)
Microbleed 10 - ROI Shape: (10, 10, 10)

NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V2.nii.gz
Microbleed Coordinates: [(69, 204, 45), (89, 182, 50), (103, 163, 70), (133, 127, 49), (66, 136, 56), (39, 104, 36), (50, 176, 33), (79, 66, 52), (88, 182, 16), (53, 112, 62)]
SWI Image Shape: (176, 256, 80)
Microbleed 1 - ROI Shape: (10, 10, 10)
Microbleed 2 - ROI Shape: (10, 10, 10

In [1]:
import pandas as pd
import os
import numpy as np
import nibabel as nib
from skimage.transform import resize
from PIL import Image

# Define a function to extract ROIs around microbleed locations and save them as PNG images
def extract_roi_and_save(swi_data, coordinates, roi_size, save_folder, target_size):
    for i, coord in enumerate(coordinates):
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]

        # Scale the ROI to [0, 255] range and convert to uint8
        roi_scaled = ((roi - np.min(roi)) / (np.max(roi) - np.min(roi)) * 255).astype(np.uint8)
        
        # Take a slice along the middle axis (z-axis)
        slice_index = roi_scaled.shape[2] // 2
        roi_slice = roi_scaled[:, :, slice_index]
        
        # Resize the ROI image to the target size using scikit-image
        roi_image_resized = resize(roi_slice, target_size, anti_aliasing=True)

        # Convert resized ROI to PIL image
        roi_image_resized = (roi_image_resized * 255).astype(np.uint8)
        roi_image = Image.fromarray(roi_image_resized)
        
        # Save the resized ROI image as PNG
        roi_image_path = os.path.join(save_folder, f'roi_{i}.png')
        roi_image.save(roi_image_path)

# Load the Excel file containing microbleed coordinates
excel_file = 'V1_data.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'Scmb-non'  # Replace 'path_to_rCMB_folder' with your folder path

# Path to the folder where you want to save the ROI images
save_folder = 'roi_images'

# Iterate through rows to load NIFTI files and extract microbleed locations
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        print(f"NIFTI Filename: {nifti_filename}")
        print(f"Microbleed Coordinates: {microbleed_coordinates}")
        print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Define the target size for the resized ROI images
        target_size = (200, 200)  # Adjust this according to your desired size
        
        # Extract ROIs around microbleed locations and save them as PNG images
        extract_roi_and_save(swi_data, microbleed_coordinates, roi_size, save_folder, target_size)

        print()  # Separate output for each NIFTI file
    else:
        print(f"NIFTI file {nifti_filename} not found in the folder.")

C:\Users\Dell\AppData\Local\Temp\ipykernel_13684\2013062508.py:65: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_13684\2013062508.py:66: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_13684\2013062508.py:67: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(

NIFTI Filename: 101_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(93, 167, 22), (30, 164, 45), (56, 187, 33), (46, 71, 39), (49, 184, 36), (52, 109, 16), (84, 161, 28), (96, 184, 65), (27, 154, 43), (132, 97, 19)]
SWI Image Shape: (176, 256, 80)

NIFTI Filename: 102_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(46, 170, 28), (109, 130, 37), (62, 94, 56), (51, 78, 47), (97, 121, 38), (75, 149, 56), (111, 128, 34), (113, 206, 24), (137, 114, 46), (99, 109, 61)]
SWI Image Shape: (176, 256, 80)

NIFTI Filename: 108_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(56, 188, 27), (47, 150, 41), (102, 157, 20), (64, 196, 27), (126, 170, 20), (101, 145, 61), (97, 196, 31), (46, 157, 52), (129, 163, 59), (38, 97, 44)]
SWI Image Shape: (176, 256, 80)

NIFTI Filename: 110_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(30, 138, 47), (141, 124, 40), (112, 93, 58), (50, 157, 36), (70, 164, 66), (82, 152, 60), (89, 128, 28), (76, 77, 

In [4]:
import pandas as pd
import os
import numpy as np
import nibabel as nib
from skimage.transform import resize
from PIL import Image

# Define a function to extract ROIs around microbleed locations and save them as PNG images
def extract_roi_and_save(swi_data, coordinates, roi_size, save_folder, target_size):
    for coord in coordinates:
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]

        # Scale the ROI to [0, 255] range and convert to uint8
        roi_scaled = ((roi - np.min(roi)) / (np.max(roi) - np.min(roi)) * 255).astype(np.uint8)
        
        # Take a slice along the middle axis (z-axis)
        slice_index = roi_scaled.shape[2] // 2
        roi_slice = roi_scaled[:, :, slice_index]
        
        # Resize the ROI image to the target size using scikit-image
        roi_image_resized = resize(roi_slice, target_size, anti_aliasing=True)

        # Convert resized ROI to PIL image
        roi_image_resized = (roi_image_resized * 255).astype(np.uint8)
        roi_image = Image.fromarray(roi_image_resized)
        
        # Save the resized ROI image as PNG
        roi_image_path = os.path.join(save_folder, f'roi_{coord[0]}_{coord[1]}_{coord[2]}.png')
        roi_image.save(roi_image_path)


# Load the Excel file containing microbleed coordinates
excel_file = 'Nonmicrobleed_coordinate.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'Scmb-non'  # Replace 'path_to_rCMB_folder' with your folder path

# Path to the folder where you want to save the ROI images
save_folder = 'roi_images1'

# Iterate through rows to load NIFTI files and extract microbleed locations
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        print(f"NIFTI Filename: {nifti_filename}")
        print(f"Microbleed Coordinates: {microbleed_coordinates}")
        print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Define the target size for the resized ROI images
        target_size = (200, 200)  # Adjust this according to your desired size
        
        # Extract ROIs around microbleed locations and save them as PNG images
        extract_roi_and_save(swi_data, microbleed_coordinates, roi_size, save_folder, target_size)

        print()  # Separate output for each NIFTI file
    else:
        print(f"NIFTI file {nifti_filename} not found in the folder.")


C:\Users\Dell\AppData\Local\Temp\ipykernel_13684\430720938.py:69: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_13684\430720938.py:70: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_13684\430720938.py:71: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(row

NIFTI Filename: 101_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(8, 136, 4), (9, 191, 61), (11, 80, 17), (12, 167, 13), (13, 191, 68), (14, 202, 12), (16, 128, 31), (17, 137, 45), (17, 140, 27), (18, 128, 37)]
SWI Image Shape: (176, 256, 80)

NIFTI Filename: 102_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(11, 41, 17), (20, 132, 27), (23, 108, 23), (26, 64, 38), (26, 193, 19), (27, 77, 43), (27, 102, 35), (27, 123, 22), (27, 133, 40), (29, 141, 22)]
SWI Image Shape: (176, 256, 80)

NIFTI Filename: 108_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(0, 102, 45), (0, 185, 19), (6, 158, 15), (10, 51, 26), (12, 140, 66), (15, 57, 17), (15, 160, 28), (17, 124, 32), (17, 142, 24), (18, 128, 46)]
SWI Image Shape: (176, 256, 80)

NIFTI Filename: 110_T1_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(1, 179, 25), (4, 168, 24), (5, 119, 24), (9, 134, 10), (11, 169, 24), (12, 103, 21), (13, 179, 17), (16, 151, 39), (16, 191, 28), 

In [19]:
import pandas as pd
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy import ndimage

# Define a function to extract ROIs around microbleed locations
def extract_roi(swi_data, coordinates, roi_size):
    rois = []
    for coord in coordinates:
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]
        rois.append(roi)
    return rois

# Define a function to compute Hessian matrix eigenvalues
def compute_hessian_eigenvalues(roi):
    # Apply Gaussian filter to the ROI for smoothing
    smoothed_roi = gaussian_filter(roi, sigma=1)
    
    # Compute gradients along x, y, and z directions using Sobel filters
    gradient_x = ndimage.sobel(smoothed_roi, axis=0)
    gradient_y = ndimage.sobel(smoothed_roi, axis=1)
    gradient_z = ndimage.sobel(smoothed_roi, axis=2)
    
    # Compute second derivatives along x, y, and z directions
    dxx = ndimage.sobel(gradient_x, axis=0)
    dyy = ndimage.sobel(gradient_y, axis=1)
    dzz = ndimage.sobel(gradient_z, axis=2)
    dxy = ndimage.sobel(gradient_x, axis=1)
    dxz = ndimage.sobel(gradient_x, axis=2)
    dyz = ndimage.sobel(gradient_y, axis=2)
    
    # Construct the Hessian matrix
    Hessian_matrix = np.array([
        [dxx.mean(), dxy.mean(), dxz.mean()],
        [dxy.mean(), dyy.mean(), dyz.mean()],
        [dxz.mean(), dyz.mean(), dzz.mean()]
    ])
    
    # Compute eigenvalues and eigenvectors of the Hessian matrix
    eigenvalues, eigenvectors = np.linalg.eig(Hessian_matrix)
    
    return eigenvalues, eigenvectors

# Define a function to compute Hessian shape features
def compute_hessian_shape_features(eigenvalues):
    if len(eigenvalues) != 3:
        # If the number of eigenvalues is not 3, return default values or handle the case accordingly
        print("Invalid number of eigenvalues for computing features")
        return 0, 0, 0  # Return default values or handle differently if needed
    
    # Sphericalness feature
    f_sphere = abs(eigenvalues[0]) / np.sqrt(abs(eigenvalues[1] * eigenvalues[2]))
    
    # Largest cross-section feature
    f_lc = abs(eigenvalues[1]) / abs(eigenvalues[2])
    
    # Fractional anisotropy feature
    f_fa = np.sqrt(0.5) * np.sqrt((eigenvalues[0] - eigenvalues[1])**2 + 
                                  (eigenvalues[1] - eigenvalues[2])**2 + 
                                  (eigenvalues[0] - eigenvalues[2])**2) / np.sqrt((eigenvalues[0]**2 + eigenvalues[1]**2 + eigenvalues[2]**2))
    
    return f_sphere, f_lc, f_fa

# Load the Excel file containing microbleed coordinates
excel_file = 'NoCMB.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'noncmb'  # Replace 'path_to_rCMB_folder' with your folder path

# Iterate through rows to load NIFTI files and extract microbleed locations
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        print(f"NIFTI Filename: {nifti_filename}")
        print(f"Microbleed Coordinates: {microbleed_coordinates}")
        print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Extract ROIs around microbleed locations
        microbleed_rois = extract_roi(swi_data, microbleed_coordinates, roi_size)

        # Compute Hessian matrix and eigenvalues for each ROI
        eigenvalues_list = []
        for roi in microbleed_rois:
            eigenvalues = compute_hessian_eigenvalues(roi)
            eigenvalues_list.append(eigenvalues)

        # Compute Hessian shape features for each microbleed
        hessian_shape_features_microbleed = []
        for i, eigenvals_tuple in enumerate(eigenvalues_list):
            eigenvals = eigenvals_tuple[0]  # Unpack the nested structure
            if len(eigenvals) != 3:
                # Print the invalid eigenvalues and their count for inspection
                print(f"Invalid eigenvalues at index {i+1}: {eigenvals} (Count: {len(eigenvals)})")
                continue
            
            f_sphere, f_lc, f_fa = compute_hessian_shape_features(eigenvals)
            hessian_shape_features_microbleed.append((f_sphere, f_lc, f_fa))

        # Display the computed Hessian shape features for each microbleed
        for i, features in enumerate(hessian_shape_features_microbleed):
            print(f"Microbleed {i+1} - Sphericalness: {features[0]:.4f}, Largest Cross-Section: {features[1]:.4f}, Fractional Anisotropy: {features[2]:.4f}")

        print()  # Separate output for each NIFTI file
    else:
        print(f"NIFTI file {nifti_filename} not found in the folder.")


C:\Users\Dell\AppData\Local\Temp\ipykernel_8428\1775931756.py:97: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_8428\1775931756.py:98: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_8428\1775931756.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(row

NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(62, 105, 53), (51, 133, 60), (131, 121, 21), (43, 149, 61), (105, 93, 51), (103, 208, 27), (52, 150, 40), (46, 181, 17), (116, 86, 50), (143, 129, 42)]
SWI Image Shape: (176, 256, 80)
Microbleed 1 - Sphericalness: 6.3611, Largest Cross-Section: 0.1785, Fractional Anisotropy: 1.1671
Microbleed 2 - Sphericalness: 2.1879, Largest Cross-Section: 0.4152, Fractional Anisotropy: 1.2247
Microbleed 3 - Sphericalness: 1.7742, Largest Cross-Section: 0.0645, Fractional Anisotropy: 1.1593
Microbleed 4 - Sphericalness: 6.1228, Largest Cross-Section: 4.4252, Fractional Anisotropy: 1.1226
Microbleed 5 - Sphericalness: 3.5465, Largest Cross-Section: 0.2554, Fractional Anisotropy: 1.2109
Microbleed 6 - Sphericalness: 32.0422, Largest Cross-Section: 0.0465, Fractional Anisotropy: 0.9305
Microbleed 7 - Sphericalness: 1.1309, Largest Cross-Section: 0.4502, Fractional Anisotropy: 1.1686
Microbleed 8 - Sphericalness: 15.3420, 

In [20]:
import pandas as pd
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy import ndimage

X = []
# Define a function to extract ROIs around microbleed locations
def extract_roi(swi_data, coordinates, roi_size):
    rois = []
    for coord in coordinates:
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]
        rois.append(roi)
    return rois

# Define a function to compute Hessian matrix eigenvalues
def compute_hessian_eigenvalues(roi):
    # Apply Gaussian filter to the ROI for smoothing
    smoothed_roi = gaussian_filter(roi, sigma=1)
    
    # Compute gradients along x, y, and z directions using Sobel filters
    gradient_x = ndimage.sobel(smoothed_roi, axis=0)
    gradient_y = ndimage.sobel(smoothed_roi, axis=1)
    gradient_z = ndimage.sobel(smoothed_roi, axis=2)
    
    # Compute second derivatives along x, y, and z directions
    dxx = ndimage.sobel(gradient_x, axis=0)
    dyy = ndimage.sobel(gradient_y, axis=1)
    dzz = ndimage.sobel(gradient_z, axis=2)
    dxy = ndimage.sobel(gradient_x, axis=1)
    dxz = ndimage.sobel(gradient_x, axis=2)
    dyz = ndimage.sobel(gradient_y, axis=2)
    
    # Construct the Hessian matrix
    Hessian_matrix = np.array([
        [dxx.mean(), dxy.mean(), dxz.mean()],
        [dxy.mean(), dyy.mean(), dyz.mean()],
        [dxz.mean(), dyz.mean(), dzz.mean()]
    ])
    
    # Compute eigenvalues and eigenvectors of the Hessian matrix
    eigenvalues, eigenvectors = np.linalg.eig(Hessian_matrix)
    
    return eigenvalues, eigenvectors

# Define a function to compute Hessian shape features
def compute_hessian_shape_features(eigenvalues):
    if len(eigenvalues) != 3:
        # If the number of eigenvalues is not 3, return default values or handle the case accordingly
        print("Invalid number of eigenvalues for computing features")
        return 0, 0, 0  # Return default values or handle differently if needed
    
    # Sphericalness feature
    f_sphere = abs(eigenvalues[0]) / np.sqrt(abs(eigenvalues[1] * eigenvalues[2]))
    
    # Largest cross-section feature
    f_lc = abs(eigenvalues[1]) / abs(eigenvalues[2])
    
    # Fractional anisotropy feature
    f_fa = np.sqrt(0.5) * np.sqrt((eigenvalues[0] - eigenvalues[1])**2 + 
                                  (eigenvalues[1] - eigenvalues[2])**2 + 
                                  (eigenvalues[0] - eigenvalues[2])**2) / np.sqrt((eigenvalues[0]**2 + eigenvalues[1]**2 + eigenvalues[2]**2))
    
    return f_sphere, f_lc, f_fa

# Load the Excel file containing microbleed coordinates
excel_file = 'NoCMB.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'noncmb'  # Replace 'path_to_rCMB_folder' with your folder path

# Iterate through rows to load NIFTI files and extract microbleed features
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        print(f"NIFTI Filename: {nifti_filename}")
        print(f"Microbleed Coordinates: {microbleed_coordinates}")
        print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Extract ROIs around microbleed locations
        microbleed_rois = extract_roi(swi_data, microbleed_coordinates, roi_size)

        # Compute Hessian matrix and eigenvalues for each ROI
        eigenvalues_list = []
        for roi in microbleed_rois:
            eigenvalues = compute_hessian_eigenvalues(roi)
            eigenvalues_list.append(eigenvalues)

        # Compute Hessian shape features for each microbleed
        hessian_shape_features_microbleed = []
        for i, eigenvals_tuple in enumerate(eigenvalues_list):
            eigenvals = eigenvals_tuple[0]  # Unpack the nested structure
            if len(eigenvals) != 3:
                # Print the invalid eigenvalues and their count for inspection
                print(f"Invalid eigenvalues at index {i+1}: {eigenvals} (Count: {len(eigenvals)})")
                continue
            
            f_sphere, f_lc, f_fa = compute_hessian_shape_features(eigenvals)
            hessian_shape_features_microbleed.append((f_sphere, f_lc, f_fa))

        # Append the computed features to the main feature list (X)
        X.extend(hessian_shape_features_microbleed)

# Convert the list of microbleed features to a NumPy array
X0 = np.array(X)

# Display the shape of the resulting feature array
print(f"Shape of X (microbleed features): {X0.shape}")

NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V1.nii.gz
Microbleed Coordinates: [(62, 105, 53), (51, 133, 60), (131, 121, 21), (43, 149, 61), (105, 93, 51), (103, 208, 27), (52, 150, 40), (46, 181, 17), (116, 86, 50), (143, 129, 42)]
SWI Image Shape: (176, 256, 80)


C:\Users\Dell\AppData\Local\Temp\ipykernel_8428\3433976206.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_8428\3433976206.py:101: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_8428\3433976206.py:102: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(

NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V2.nii.gz
Microbleed Coordinates: [(69, 204, 45), (89, 182, 50), (103, 163, 70), (133, 127, 49), (66, 136, 56), (39, 104, 36), (50, 176, 33), (79, 66, 52), (88, 182, 16), (53, 112, 62)]
SWI Image Shape: (176, 256, 80)
NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V3.nii.gz
Microbleed Coordinates: [(31, 144, 49), (116, 136, 58), (53, 147, 53), (139, 106, 45), (33, 121, 32), (140, 133, 38), (79, 84, 43), (61, 75, 52), (124, 178, 27), (48, 119, 39)]
SWI Image Shape: (176, 256, 80)
NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V4.nii.gz
Microbleed Coordinates: [(96, 128, 54), (137, 102, 33), (99, 80, 44), (121, 180, 52), (49, 72, 34), (102, 104, 58), (68, 183, 59), (76, 204, 41), (104, 70, 36), (75, 208, 53)]
SWI Image Shape: (176, 256, 80)
NIFTI Filename: 14_T4_MRI_SWI_BFC_50mm_HM_sCMB_V5.nii.gz
Microbleed Coordinates: [(124, 192, 20), (41, 66, 36), (63, 162, 17), (79, 66, 52), (93, 174, 47), (96, 128, 54), (125, 189, 19), (23, 129, 28), (1

In [21]:
# Assuming X contains the microbleed features

# Determine the number of microbleeds in X
num_microbleeds = X0.shape[0]

# Set the labels for microbleeds as 1
Y0 = np.zeros(num_microbleeds)

# Display the shape of the label array
print(f"Shape of Y (labels): {Y0.shape}")

# Display a few labels to confirm
print(Y0)  # Displaying the first 10 labels as an example


Shape of Y (labels): (570,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0